In [1]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

import pandas as pd
# Modulo de optimizacion en scipy
from scipy import optimize

# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [3]:
# donde tienes 784 características de entrada (28*28) y 10 etiquetas posibles (num_labels = 10), 
input_layer_size  = 3072

num_labels = 10

data = pd.read_csv('data_13.csv', delimiter=',')

X = data.iloc[:, :].values
#y = data.iloc[:, 0].values

#m = y.size

In [4]:
print(X)

[[136.  70.  39. ... 255. 255. 255.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [5]:
# Normalizar dividiendo entre 255
X = X / 255.0

print(X)

[[0.53333333 0.2745098  0.15294118 ... 1.         1.         1.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [6]:
# La función sigmoidal toma cualquier valor real y lo "aplana" a un valor entre 0 y 1,
# lo cual es útil para interpretar la salida como una probabilidad.
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [127]:
# esta función es para calcular el costo y el gradiente en tu algoritmo de optimización
def lrCostFunction(theta, X, y, lambda_):
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    # Calcula la hipótesis utilizando la función sigmoidal.
    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    # Calcula el costo regularizado utilizando la función de costo de la regresión logística,
    # añadiendo el término de regularización para todos los parámetros excepto theta_0.
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [128]:
# Esta funcion entrenará un clasificador de regresión logística para cada clase utilizando el
# método de descenso del gradiente conjugado, optimizando la función de costo regularizada
# lrCostFunction para cada clase. Luego, all_theta contendrá los parámetros theta óptimos para
# cada clasificador, que puedes utilizar para hacer predicciones sobre nuevos datos.
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape

    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [129]:
# est funcion llamará a oneVsAll para entrenar los clasificadores de regresión logística
# sin regularización y luego imprimirá la forma de all_theta.
# Dependiendo de tus datos y del número de características, la salida podría ser algo como
# (10, 785) si X tiene 784 características más el término de sesgo añadido.
# Esto significa que tienes 10 clasificadores binarios entrenados,
# uno para cada clase, con 785 parámetros (incluyendo el término de sesgo).
lambda_ = 0
all_theta = oneVsAll(X, y, num_labels, lambda_)
print(all_theta.shape)

(10, 785)


In [130]:
print(all_theta)

[[-4.74495975  0.          0.         ...  0.          0.
   0.        ]
 [-1.20010331  0.          0.         ...  0.          0.
   0.        ]
 [-2.95016202  0.          0.         ...  0.          0.
   0.        ]
 ...
 [-1.10023019  0.          0.         ...  0.          0.
   0.        ]
 [-7.10986697  0.          0.         ...  0.          0.
   0.        ]
 [-4.50093645  0.          0.         ...  0.          0.
   0.        ]]


In [131]:
# Después de entrenar los clasificadores utilizando oneVsAll y obtener all_theta,
# puedes utilizar predictOneVsAll para predecir las etiquetas de clase para nuevos datos X
# all_theta contendrá los parámetros theta óptimos para cada clasificador binario entrenado
# utilizando el enfoque uno contra todos. Luego, predictOneVsAll utilizará estos parámetros
# para predecir las etiquetas de clase para el conjunto de datos X.
# Las predicciones se almacenarán en predictions
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Add ones to the X data matrix
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [132]:
# La línea print('Precisión del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
# muestra la precisión del modelo en el conjunto de entrenamiento.
# Esto te da una idea de qué tan bien está aprendiendo tu modelo sobre los datos de entrenamiento.
# Las líneas relacionadas con XPrueba muestran cómo puedes usar el modelo entrenado
# (all_theta) para hacer predicciones sobre nuevos datos (XPrueba). La comparación de p
# (predicciones) con y[10:150] (etiquetas verdaderas) te permite evaluar cómo se desempeña el modelo en datos no vistos.
print(X.shape)
pred = predictOneVsAll(all_theta, X)
print('Precision del conjuto de entrenamiento: {:.2f}%'.format(np.mean(pred == y) * 100))
XPrueba = X[10:150, :].copy()
print(XPrueba.shape)
XPrueba = np.concatenate([np.ones((140, 1)), XPrueba], axis=1)
print(XPrueba.shape)

# Realiza predicciones sobre XPrueba utilizando los parámetros all_theta entrenados.
# sigmoid(XPrueba.dot(all_theta.T)) calcula las probabilidades de clase para cada ejemplo en XPrueba,
# y np.argmax(..., axis=1) determina la clase predicha tomando el índice con la probabilidad más alta.
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis = 1)
print(p)

print(y[10:150])

(60000, 784)
Precision del conjuto de entrenamiento: 92.20%
(140, 784)
(140, 785)
[3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 7 7 3 8 6 7 0 5 6 0 7 6 1 8 7 9 3 9 8
 5 5 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6 3 0 2 1 1 7 0 0 2 6
 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1 5 7 1 7 1 1 6 3 0 2 9 3 1 1 0 4 9 2 0 0 2
 0 2 7 1 8 6 4 1 6 3 4 3 9 5 3 3 8 5 4 7 7 4 2 8 5 8 6 2 3]
[3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6 0 7 6 1 8 7 9 3 9 8
 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6 3 0 2 1 1 7 9 0 2 6
 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1 5 7 1 7 1 1 6 3 0 2 9 3 1 1 0 4 9 2 0 0 2
 0 2 7 1 8 6 4 1 6 3 4 5 9 1 3 3 8 5 4 7 7 4 2 8 5 8 6 7 3]
